# Overview

# Crawl data

In [46]:
#import
import requests
import re
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np


In [2]:
#Crawl URL của 3000 sản phẩm và lưu vào file links.txt
def get_id(url):

    # Mở file để ghi
    with open('data/links.txt', 'w') as f:

        # Mỗi trang xe có link của 20 xe
        for i in range(1, 151):
            print(i)
            response = requests.get(url.format(i))

            # Trích xuất phần cần thiết của mã nguồn trang web
            text = response.text

            # Tìm tất cả các cặp "id" và "slug" trong phần đã trích xuất
            matches = re.findall(r'"id":"(\d{8})","slug":"([\w-]+)",', text)

            for match in matches:
                id, slug = match
                # Ghi vào file thay vì in ra
                f.write(f'https://carvago.com/car/{id}/{slug}\n')

# Sử dụng hàm với url đã được chỉnh sửa để chứa số trang
#get_id('https://carvago.com/cars?page={}')

# Crawl thông tin của từng sản phẩm chi tiết

In [39]:
def get_data(text, pattern):
    """
    Trả về tuple chứa các thông tin được trích xuất từ text theo pattern
    """
    match = re.search(pattern, text)
    if match:
        return match.groups()
    else:
        return None, None, None, None 

def extract_car_info(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            if (soup.title.text == 'Carvago.com | Buying a used car online. Warranty and home delivery.'):
                print('web nolonger available, car has been sold!!!!!')
                return None
            print('200 OK')

            car_info_divs = soup.find_all('div', class_='css-1b3bvbl')  # Adjust the class if needed

            carname = ""

            for car_info_div in car_info_divs:
                attribute_name_div = car_info_div.find('div', class_='css-f6f0k1')
                if attribute_name_div is not None:
                    attribute_name = attribute_name_div.text.strip()

                    if attribute_name == 'Model':
                        car_link_div = car_info_div.find('div', class_='css-qres8i')
                        if car_link_div is not None:
                            car_link_tag = car_link_div.find('a')
                            if car_link_tag is not None:
                                car_link = car_link_tag['href']
                                carname = car_link.split('/')[2:]  # Extract the last part of the URL
                                carname = " ".join(carname)
            
            # Lấy tên thông tin (ví dụ: 'Make', 'Model', ...)
            keys = []
            keys.append('CARNAME')
            keys.append('ID')
            keys.extend([div.get_text() for div in soup.find_all('div', {'class': 'css-f6f0k1'})])

            # Lấy giá trị thông tin (ví dụ: 'Chevrolet', 'Orlando', ...)
            values = []
            values.append(carname)
            values.append(url.split('/')[4])
            for key, div in zip(keys[2: ], soup.find_all('div', {'class': 'css-1b3bvbl'})):
                if key == 'VIN':
                    if div.find('div', {'class': 'css-10odsig'}) is not None:
                        value = div.find('div', {'class': 'css-10odsig'}).get_text()
                    else:
                        value = div.find('div', {'class': 'css-1yu0dva'}).get_text()
                else:
                    value = div.find('div', {'class': 'css-qres8i'}).get_text()
                values.append(value)

            consumption = [div.get_text() for div in soup.find_all('div', {'class': 'css-c55dw0'})]
            keys.append('Consumption')
            values.append('; '.join(consumption))

            # Lấy giá tiền và đơn vị tiền tệ
            price_pattern = r'"price":(\d+),"price_currency":\{"id":(\d+),"name":"(.*?)","const_key":"CURRENCY_(.*?)"\}'
            price, currency_id, currency_name, currency_const_key = get_data(response.text, price_pattern)
            # if price, currency_id, currency_name, currency_const_key is not None:
            #     keys.append('Price')
            #     values.append(price)
            #     keys.append('Currency')
            #     values.append(currency_name)
            if price is not None:
                keys.append('Price')
                values.append(price)

            if currency_name is not None:
                keys.append('Currency')
                values.append(currency_name)

            # Lấy các thẻ
            tags_pattern = r'"featured_tags":(\[.*?\])'
            featured_tags = json.loads(get_data(response.text, tags_pattern)[0])
            keys.append('Tags')
            #Chuyển các giá trị trong các thẻ thành một string
            values.append('; '.join([tag['name'] for tag in featured_tags]))

            df = pd.DataFrame([values], columns=keys)

            return df
                
    except requests.exceptions.RequestException as e:
        print(f"Failed to get the webpage at {url} due to error: {e}")

In [54]:
# Read links from data.txt
with open('data/link2.txt', 'r') as file:
    links = file.read().splitlines()

list_dataframe = []
data = pd.DataFrame()
# Loop through each link and extract information
i = 0
for link in links:
    print(i, '/', len(links))
    car_info = extract_car_info(link)
    if (car_info is None):
        print(f"Skipping link due to error")
        continue  
    list_dataframe.append(car_info)


temp_df = list_dataframe.copy()
    


0 / 1500
Skipping link due to error
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
Skipping link due to error
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
Skipping link due to error
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
200 OK
0 / 1500
Skipping link due to error
0 / 1500
Skipping link due to error
0 / 1500
Skipping link due t

In [56]:


def rename_duplicates(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

list_dataframe = [rename_duplicates(df) for df in list_dataframe]

# Get a union of all DataFrame columns
all_columns = set().union(*[set(df.columns) for df in list_dataframe])

# Make sure all DataFrames have all columns, fill missing ones with NaN
list_dataframe = [df.reindex(columns=all_columns, fill_value=np.nan) for df in list_dataframe]

# Now you can concatenate
crawldata = pd.concat(list_dataframe, ignore_index=True)







In [57]:
crawldata.shape

(1099, 45)

# Lưu vào data/products.csv

In [58]:
crawldata.to_csv("data/sell_data2.csv",header=True)
# crawldata.shape